In [ ]:
%matplotlib inline
%config Application.log_level="INFO"

import matplotlib
import numpy as np
import mxnet as mx

import matplotlib.pyplot as plt

In [ ]:
import logging
logging.basicConfig()
logger = logging.getLogger(__name__)
logger.setLevel(logging.DEBUG)

In [ ]:
def load_data(file):
    data = np.loadtxt(file, delimiter=',', skiprows=1)
    return data

def split_data(data, ratio):
    rows = data.shape[0]
    rows = int(rows * ratio)
    train_data, eval_data = np.split(data, [rows,], axis=0)
    return train_data, eval_data

def data_to_iter(data, batch_size=10):
    y, x = np.split(data, [1], axis=1)
    
    y = y.flatten()
    
    data_iter = mx.io.NDArrayIter(
        data=x,
        label=y,
        batch_size=batch_size,
        last_batch_handle='drop',
    )
    
    return data_iter


In [ ]:
def build_model(context):
    data = mx.sym.Variable('data')
    label = mx.sym.Variable('softmax_label')
    data = mx.sym.reshape(data=data, shape=[-1, 28, 28])
    
    net = mx.sym.FullyConnected(data, num_hidden=64)
    net = mx.sym.Activation(net, act_type='relu')
    net = mx.sym.FullyConnected(net, num_hidden=10)
    net = mx.sym.SoftmaxOutput(net, label, preserve_shape=True, name='softmax')
    
    model = mx.mod.Module(
        symbol=net,
        context=context,
        data_names=['data'],
        label_names=['softmax_label'],
        logger=logger,
    )
    
    return model

In [ ]:
def best_prediction(predictions):
    best_prediction = max(
        enumerate(predictions), 
        key = lambda x: x[1]
    )
    label = best_prediction[0]
    
    return label
    

def visualize_prediction(datum, predictions):
    label = int(datum[0])
    datum = datum[1:]
    
    image = datum.reshape(28, 28)
    
    prediction = best_prediction(predictions)
    
    plt.title("Label: {}, Prediction: {}".format(label, prediction))
    plt.imshow(image, cmap='gray')
    plt.show()

In [ ]:
data = load_data('train.csv')


In [ ]:
train_data, eval_data = split_data(data, 0.8)

train_iter = data_to_iter(train_data)
eval_iter = data_to_iter(eval_data)

print("Training data shape:", train_data.shape)
print("Evaluation data shape:", eval_data.shape)


In [ ]:
model = build_model(mx.cpu())

In [ ]:
""" 
Predicting with an untrained model

The model (mx.Module) needs to be bound to certain data shapes
to bootstrap the symbol to proper data shapes

Then the model is initialized with normally distributed noise

Initialized model can be used to predict as well =D
"""

data_iter = data_to_iter(data)

model.bind(
    data_shapes=data_iter.provide_data,
    force_rebind=True,
    label_shapes=data_iter.provide_label,
    for_training=False,
)

model.init_params(
    mx.initializer.Normal(),
    force_init=True,
)

model.score(data_iter, ['loss', 'acc'])

In [ ]:
model.fit(
    train_data=train_iter,
    eval_data=eval_iter,
    optimizer='sgd',
    optimizer_params={'learning_rate':0.001},
    eval_metric=['acc'],
    num_epoch=10,
    force_init=True,
    force_rebind=True,
)

In [ ]:
model.score(data_iter, ['loss', 'acc'])

In [ ]:
predictions = model.predict(data_iter)

In [ ]:
for i in range(100, 120):
    visualize_prediction(data[i], predictions[i])
